In [2]:
import sys
print(sys.executable)
import torch
print(torch.cuda.is_available())

c:\Users\Tesnim\miniconda3\envs\yoloproj_env_new\python.exe
True


In [1]:
import cv2
from ultralytics import YOLO
import supervision as sv
from supervision import BoxAnnotator, LabelAnnotator
import numpy as np

In [5]:
#video ylu 
video_path="videos/v2.mp4"
output_video_path = "output/street_detection.mp4"
#YOLOV8 nano modeli yükle 
model=YOLO('yolov8n.pt')

#video dosyasını oku 
cap=cv2.VideoCapture(video_path)

#video başarılı bir şekilde açıldı mı kontrol et 
if not cap.isOpened():
    #print("Hata:Video dosyası açılamadı veya yolu yanlış:{video_path}")
    raise FileNotFoundError(f"Video bulunamadı: {video_path}") 
else:
    #video özelliklerini çıkar 
    width =int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps=int(cap.get(cv2.CAP_PROP_FPS))
    frame_count=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if fps == 0.0:
        fps = 30.0  # FPS alınamadıysa yedek olarak 30 ver

    print(f"Video başarıyla açıldı : {video_path}")
    print(f"Video boyutları : {width}x{height}")
    print(f"FPS:{fps}")
    print(f"Toplam Kare Sayısı : {frame_count}")


Video başarıyla açıldı : videos/v2.mp4
Video boyutları : 1920x1080
FPS:25
Toplam Kare Sayısı : 341


In [6]:
from supervision import BoxAnnotator
import os

os.makedirs("debug", exist_ok=True)  # Debug klasörü oluştur
os.makedirs(os.path.dirname("output/street_detection.mp4"), exist_ok=True)  # Output dizini oluştur

box_annotator = BoxAnnotator(
    thickness=2,                  # Kutu çizgi kalınlığı
    color=sv.Color(255, 0, 0)    # Kırmızı renk kutu çizgisi
)

label_annotator = LabelAnnotator(
    text_thickness=2,             # Yazı kalınlığı
    text_scale=1,                 # Yazı boyutu
    text_color=sv.Color(255, 255, 255),  # Beyaz yazı rengi
    color=sv.Color(0, 0, 0),     # Siyah arka plan
    text_padding=5                # Yazı etrafı boşluk
)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Video codec ayarı
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))  # Video yazıcı başlat

frame_idx = 0
MAX_FRAMES_TO_PROCESS = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Video toplam kare sayısı

while cap.isOpened():
    ret, frame = cap.read()  # Kare oku

    if not ret and frame_idx < MAX_FRAMES_TO_PROCESS:
        print("Video sonuna ulaşıldı veya kare okunamadı. Döngüden çıkılıyor.")
        break
    elif frame_idx >= MAX_FRAMES_TO_PROCESS:
        print("Maksimum kare sayısına erişildi. Döngüden çıkılıyor.")
        break
    else:
        if frame is None:
            print(f"Hata: Frame {frame_idx} boş geldi!")
            break

    results = model(frame, classes=0, conf=0.25)[0]  # Sadece insan sınıfını tespit et (classes=0)
    print(results)

    detections = sv.Detections.from_ultralytics(results)  # Tespitleri supervision formatına dönüştür
    print(f"Kare {frame_idx} - Tespit Sayısı: {len(detections)}")

    labels = [
        f"{model.names[class_id]} {confidence:.2f}"  # Sınıf ismi ve güven skorunu hazırla
        for class_id, confidence in zip(detections.class_id, detections.confidence)
    ]

    annotated_frame = box_annotator.annotate(scene=frame.copy(), detections=detections)  # Kutu çiz
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)  # Etiket ekle

    if annotated_frame is None or annotated_frame.size == 0:
        print("annotated_frame boş. Atlanıyor.")
        continue

    out.write(annotated_frame)  # Videoya yaz

    frame_idx += 1  # Kare sayısını artır

cap.release()  # Video kaynağını serbest bırak
out.release()  # Video yazıcıyı kapat
cv2.destroyAllWindows()  # Açık OpenCV pencerelerini kapat

if os.path.exists(output_video_path):
    size = os.path.getsize(output_video_path)
    print(f"Video işleme tamamlandı. Sonuçlar '{output_video_path}' adresine kaydedildi.")
    print(f"Oluşturulan video boyutu: {size / 1024:.2f} KB")
    if size < 1000:
        print(" Uyarı: Video dosyası çok küçük, oynatılamayabilir.")
else:
    print(" Video dosyası oluşturulamadı!")



0: 384x640 37 persons, 39.0ms
Speed: 49.5ms preprocess, 39.0ms inference, 62.1ms postprocess per image at shape (1, 3, 384, 640)
ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange'